In [2]:
import polars as pl
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
from tqdm import tqdm

# Init Faker
fake = Faker('es_CO')
np.random.seed(42)
random.seed(42)

# --- Parameters ---
n_ventas = 200000
n_clientes = 2000
n_productos = 40
n_canales = 5
n_promociones = 20
n_regiones = 10
n_inventarios = 120

start_date_data = datetime(2023, 1, 1)
end_date_data = datetime(2025, 6, 15)

# Elasticity parameters - Ajustados para mayor sensibilidad al precio
elasticidades_por_categoria = {
    'Agua': {'min': -0.8, 'max': -1.2},           # Más sensible al precio
    'Gaseosa': {'min': -1.0, 'max': -1.5},        # Más sensible al precio
    'Jugo': {'min': -1.2, 'max': -1.8},           # Más sensible al precio
    'Bebida de Té': {'min': -1.5, 'max': -2.2},   # Más sensible al precio
    'Bebida Energética': {'min': -2.0, 'max': -3.0}  # Muy sensible al precio
}
# Reducida la probabilidad de elasticidad positiva para casos excepcionales
prob_elasticidad_positiva = 0.01
rango_elasticidad_positiva = {'min': 0.05, 'max': 0.2}

# --- NUEVOS PARÁMETROS DE CONTROL DE FLUCTUACIÓN POR PRODUCTO (AJUSTADOS) ---
MAX_PERCENT_CHANGE_PER_MONTH_PRODUCT_AGGREGATE = 0.12  # Aumentado para permitir más variación
MIN_QUANTITY_PER_SALE = 1
MAX_QUANTITY_PER_SALE = 24
DAILY_QUANTITY_FLUCTUATION_FACTOR = 0.25  # Aumentado para más variación diaria

# --- SEASONALITY ADJUSTMENT - Reforzado para meses cálidos y diciembre ---
seasonal_multipliers = {
    1: random.uniform(0.85, 0.90),   # Jan (Post-holiday drop)
    2: random.uniform(0.80, 0.85),   # Feb (Low season)
    3: random.uniform(0.85, 0.90),   # Mar
    4: random.uniform(0.90, 0.95),   # Apr
    5: random.uniform(0.95, 1.0),    # May (Preparing for summer)
    6: random.uniform(1.25, 1.35),   # Jun (Strong summer season start)
    7: random.uniform(1.35, 1.45),   # Jul (Peak summer season)
    8: random.uniform(1.30, 1.40),   # Aug (Strong summer season)
    9: random.uniform(0.95, 1.0),    # Sep (Post-summer normalization)
    10: random.uniform(0.85, 0.90),  # Oct
    11: random.uniform(0.95, 1.0),   # Nov (Preparing for holidays)
    12: random.uniform(1.35, 1.45)   # Dec (Strong holiday season)
}

# --- PROMOTIONAL IMPACT PARAMETERS (NUEVOS) ---
PROMOTION_ELASTICITY_MULTIPLIER = 1.5  # Multiplicador del efecto de elasticidad durante promociones
MIN_PROMOTION_BOOST = 1.2  # Mínimo impulso a las ventas durante promociones
MAX_PROMOTION_BOOST = 1.8  # Máximo impulso a las ventas durante promociones

# --- 1. Regions Table ---
ciudades_colombia = [
    {"nombre_region": "Cundinamarca", "ciudad": "Bogotá", "latitud": 4.7110, "longitud": -74.0721},
    {"nombre_region": "Antioquia", "ciudad": "Medellín", "latitud": 6.2442, "longitud": -75.5812},
    {"nombre_region": "Valle del Cauca", "ciudad": "Cali", "latitud": 3.4516, "longitud": -76.5320},
    {"nombre_region": "Atlántico", "ciudad": "Barranquilla", "latitud": 10.9685, "longitud": -74.7813},
    {"nombre_region": "Bolívar", "ciudad": "Cartagena", "latitud": 10.3910, "longitud": -75.4794},
    {"nombre_region": "Santander", "ciudad": "Bucaramanga", "latitud": 7.1254, "longitud": -73.1198},
    {"nombre_region": "Norte de Santander", "ciudad": "Cúcuta", "latitud": 7.8939, "longitud": -72.5078},
    {"nombre_region": "Tolima", "ciudad": "Ibagué", "latitud": 4.4389, "longitud": -75.2114},
    {"nombre_region": "Meta", "ciudad": "Villavicencio", "latitud": 4.1415, "longitud": -73.6268},
    {"nombre_region": "Boyacá", "ciudad": "Tunja", "latitud": 5.5352, "longitud": -73.3677}
]

regiones_data = []
for i in range(n_regiones):
    region_info = ciudades_colombia[i] if i < len(ciudades_colombia) else random.choice(ciudades_colombia)
    regiones_data.append({
        'region_id': i + 1,
        'nombre_region': region_info['nombre_region'],
        'ciudad': region_info['ciudad'],
        'latitud': region_info['latitud'],
        'longitud': region_info['longitud']
    })
df_regiones = pl.DataFrame(regiones_data)

# Region weights
pesos_regiones = {
    "Bogotá": 0.25, "Medellín": 0.20, "Cali": 0.15, "Barranquilla": 0.15,
    "Cartagena": 0.10, "Bucaramanga": 0.05, "Cúcuta": 0.04,
    "Ibagué": 0.03, "Villavicencio": 0.02, "Tunja": 0.01
}
prob_regiones_para_seleccion = np.array([pesos_regiones.get(row['ciudad'], 0.01) for row in df_regiones.iter_rows(named=True)])
prob_regiones_para_seleccion /= prob_regiones_para_seleccion.sum()
regiones_para_seleccion = df_regiones['region_id'].to_list()

# --- 2. Clients Table ---
ciudades_regiones = df_regiones['ciudad'].to_list()

# Calcular fechas límite para la distribución de última compra
fecha_actual = end_date_data
fecha_365_dias_atras = fecha_actual - timedelta(days=365)

# Determinar el número exacto de clientes inactivos (entre 5% y 15%)
porcentaje_inactivos = random.uniform(0.05, 0.15)
n_clientes_inactivos = int(n_clientes * porcentaje_inactivos)
n_clientes_activos = n_clientes - n_clientes_inactivos

clientes = []
for i in range(1, n_clientes + 1):
    # Determinar si este cliente será inactivo
    es_inactivo = i <= n_clientes_inactivos
    
    if es_inactivo:
        # Para clientes inactivos, última compra entre 365 y 730 días atrás
        ultima_compra = fake.date_between(
            start_date=fecha_actual - timedelta(days=730),
            end_date=fecha_365_dias_atras
        )
    else:
        # Para clientes activos, última compra en los últimos 365 días
        ultima_compra = fake.date_between(
            start_date=fecha_365_dias_atras,
            end_date=fecha_actual
        )
    
    clientes.append({
        'cliente_id': i,
        'nombre': fake.name(),
        'edad': np.random.randint(18, 80),
        'genero': np.random.choice(['M', 'F']),
        'ciudad': random.choice(ciudades_regiones),
        'frecuencia_compra': np.random.randint(1, 20),
        'ultima_compra': ultima_compra
    })
df_clientes = pl.DataFrame(clientes)

# --- 3. Products Table ---
volumenes_gaseosa_jugo_ml = [250, 600, 1000, 2000]
unidades_por_caja_gaseosa_jugo = [6, 12, 24]
volumenes_energia_ml = [250, 500]
unidades_por_caja_energia = [4, 6]
volumenes_agua_ml = [500, 1000, 2000, 5000]
unidades_por_caja_agua = [1, 6, 12]

categorias_marcas_sabor_base = [
    {'categoria_base': 'Gaseosa', 'sabores': ['Cola', 'Naranja', 'Limón', 'Piña', 'Manzanita', 'Uva'], 'marcas': ['Zulianita', 'Competidor1']},
    {'categoria_base': 'Bebida de Té', 'sabores': ['Té Negro', 'Té Verde'], 'marcas': ['Zulianita', 'Competidor2']},
    {'categoria_base': 'Jugo', 'sabores': ['Jugo Naranja', 'Jugo Manzana', 'Jugo Lima'], 'marcas': ['Zulianita', 'Competidor1']},
    {'categoria_base': 'Bebida Energética', 'sabores': ['Energía Extrema', 'Power Up'], 'marcas': ['Zulianita', 'Competidor2']},
    {'categoria_base': 'Agua', 'sabores': ['Agua con Gas', 'Agua Sin Gas'], 'marcas': ['Zulianita', 'Competidor1']}
]

productos = []
for i in range(n_productos):
    tipo_prod_info = random.choice(categorias_marcas_sabor_base)
    categoria = tipo_prod_info['categoria_base']
    sabor = random.choice(tipo_prod_info['sabores'])
    marca = random.choice(tipo_prod_info['marcas'])

    nombre_producto_str = sabor
    volumen_ml_val = 0
    unidades_caja_val = 1

    if categoria in ['Gaseosa', 'Jugo']:
        volumen_ml_val = random.choice(volumenes_gaseosa_jugo_ml)
        unidades_caja_val = random.choice(unidades_por_caja_gaseosa_jugo)
        nombre_producto_str = f"{sabor} {volumen_ml_val}mL x {unidades_caja_val}uds"
    elif categoria == 'Bebida Energética':
        volumen_ml_val = random.choice(volumenes_energia_ml)
        unidades_caja_val = random.choice(unidades_por_caja_energia)
        nombre_producto_str = f"{sabor} {volumen_ml_val}mL x {unidades_caja_val}uds"
    elif categoria == 'Agua':
        volumen_ml_val = random.choice(volumenes_agua_ml)
        unidades_caja_val = random.choice(unidades_por_caja_agua)
        nombre_producto_str = f"{sabor} {volumen_ml_val // 1000}L x {unidades_caja_val}uds" if volumen_ml_val >= 1000 else f"{sabor} {volumen_ml_val}mL x {unidades_caja_val}uds"
    elif categoria == 'Bebida de Té':
        volumen_ml_val = random.choice([300, 500, 1000])
        unidades_caja_val = random.choice([1, 6, 12])
        nombre_producto_str = f"{sabor} {volumen_ml_val}mL x {unidades_caja_val}uds"

    productos.append({
        'producto_id': i + 1,
        'nombre_producto': nombre_producto_str,
        'categoria': categoria,
        'marca': marca,
        'volumen_ml_base': volumen_ml_val,
        'unidades_caja_base': unidades_caja_val
    })
df_productos = pl.DataFrame(productos)

# --- 4. Price History Table ---
historico_precios = []
hist_precio_id_counter = 1

precios_por_ml_base_categoria = {
    'Agua': 2.5, 'Gaseosa': 3.5, 'Jugo': 4.0, 'Bebida Energética': 10.0, 'Bebida de Té': 6.0
}
costos_por_ml_base_categoria = {
    'Agua': 1.0, 'Gaseosa': 1.8, 'Jugo': 2.2, 'Bebida Energética': 4.5, 'Bebida de Té': 3.0
}

for producto_row in df_productos.iter_rows(named=True):
    current_date = start_date_data.replace(day=1)

    volumen_para_calculo = producto_row['volumen_ml_base'] if producto_row['volumen_ml_base'] > 0 else 1000
    unidades_para_calculo = producto_row['unidades_caja_base'] if producto_row['unidades_caja_base'] > 0 else 1
    categoria = producto_row['categoria']

    base_precio_ml_actual = precios_por_ml_base_categoria.get(categoria, 3.0)
    base_costo_ml_actual = costos_por_ml_base_categoria.get(categoria, 1.8)

    initial_precio_unitario = round(base_precio_ml_actual * volumen_para_calculo * unidades_para_calculo, 2)
    initial_costo_variable = round(base_costo_ml_actual * volumen_para_calculo * unidades_para_calculo, 2)

    while current_date <= end_date_data:
        historico_precios.append({
            'historico_precio_id': hist_precio_id_counter,
            'producto_id': producto_row['producto_id'],
            'fecha_actualizacion': current_date.date(),
            'precio_base': initial_precio_unitario,
            'costo_variable': initial_costo_variable
        })
        hist_precio_id_counter += 1

        incremento_porcentaje_precio = random.uniform(0.00, 0.02)
        incremento_porcentaje_costo = random.uniform(0.00, 0.015)

        initial_precio_unitario = round(initial_precio_unitario * (1 + incremento_porcentaje_precio), 2)
        initial_costo_variable = round(initial_costo_variable * (1 + incremento_porcentaje_costo), 2)

        if initial_costo_variable >= initial_precio_unitario:
            initial_costo_variable = round(initial_precio_unitario * 0.7, 2)

        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year + 1, month=1)
        else:
            current_date = current_date.replace(month=current_date.month + 1)

df_historico_precios = pl.DataFrame(historico_precios)

# --- 5. Channels Table ---
canales = {
    'canal_id': range(1, n_canales + 1),
    'nombre_canal': ['Supermercado', 'Tienda de Conveniencia', 'E-commerce', 'Vending Machine', 'Hipermercado'],
    'tipo_canal': ['Físico', 'Físico', 'Online', 'Físico', 'Físico']
}
df_canales = pl.DataFrame(canales)

# --- 6. Promotions Table ---
promociones_data = []
for i in range(1, n_promociones + 1):
    fecha_inicio = fake.date_between(start_date=start_date_data - timedelta(days=180), end_date=end_date_data - timedelta(days=90))
    fecha_fin = fecha_inicio + timedelta(days=random.randint(30, 180))

    if fecha_fin > end_date_data.date():
        fecha_fin = end_date_data.date()

    promociones_data.append({
        'promocion_id': i,
        'nombre_promocion': f"Promo {i}",
        'descuento_porcentaje': np.random.randint(5, 30),
        'fecha_inicio': fecha_inicio,
        'fecha_fin': fecha_fin
    })
df_promociones = pl.DataFrame(promociones_data)

# --- 7. Inventory Table ---
inventarios = {
    'inventario_id': range(1, n_inventarios + 1),
    'producto_id': np.random.choice(df_productos['producto_id'], n_inventarios),
    'region_id': np.random.choice(df_regiones['region_id'], n_inventarios),
    'stock': np.random.randint(5, 200, n_inventarios),
    'fecha_actualizacion': [fake.date_between(start_date='-6m', end_date='today') for _ in range(n_inventarios)]
}
df_inventarios = pl.DataFrame(inventarios)

# --- 8. Sales Table (with seasonality and controlled monthly quantity per product) ---
sales_data_chunks = []
venta_id_counter_final = 1

# Store monthly product state for EPD calculation and target setting
last_product_monthly_state = {}   # {(producto_id, 'YYYY-MM'): {'precio': P_avg, 'cantidad': Q_total}}
monthly_product_target_quantities = {} # {(producto_id, 'YYYY-MM'): target_total_quantity for the current month}
daily_product_sales_distribution = {} # {(producto_id, 'YYYY-MM-DD'): target_daily_quantity}


# Pre-calculate product categories for faster lookup
product_category_map = {row['producto_id']: row['categoria'] for row in df_productos.select(['producto_id', 'categoria']).iter_rows(named=True)}

total_days_simulation = (end_date_data - start_date_data).days + 1

# Define the consistent schema for daily sales data upfront
daily_sales_schema = {
    'venta_id': pl.Int64,
    'fecha': pl.String,
    'cliente_id': pl.Int64,
    'producto_id': pl.Int64,
    'cantidad': pl.Int64,
    'canal_id': pl.Int64,
    'region_id': pl.Int64,
    'promocion_id': pl.Int64,
    'historico_precio_id': pl.Int64
}

# Initial monthly trend multiplier (for overall sales volume)
monthly_sales_trend_multiplier = 1.0
last_month_year = None


# Use tqdm for progress visualization
for day_offset in tqdm(range(total_days_simulation), desc="Generando datos de ventas"):
    current_sale_date = start_date_data + timedelta(days=day_offset)
    current_sale_date_str = current_sale_date.strftime('%Y-%m-%d')
    current_month_str = current_sale_date.strftime('%Y-%m')

    month = current_sale_date.month
    year = current_sale_date.year

    # --- Monthly Aggregated Quantity Control Logic ---
    if current_month_str != last_month_year:
        # 1. Update last_product_monthly_state with actual sales from the *just finished* month
        if last_month_year is not None:
            prev_month_end = current_sale_date - timedelta(days=1)
            prev_month_start = prev_month_end.replace(day=1)
            prev_month_str_for_key = prev_month_start.strftime('%Y-%m')

            # Filter sales_data_chunks for the previous month
            df_prev_month_sales_list = []
            for chunk_df in sales_data_chunks:
                # Assuming 'fecha' in chunks is always in '%Y-%m-%d' format string
                chunk_filtered = chunk_df.filter(
                    (pl.col('fecha').str.strptime(pl.Date, '%Y-%m-%d') >= prev_month_start.date()) &
                    (pl.col('fecha').str.strptime(pl.Date, '%Y-%m-%d') <= prev_month_end.date())
                )
                if not chunk_filtered.is_empty():
                    df_prev_month_sales_list.append(chunk_filtered)

            if df_prev_month_sales_list:
                df_prev_month_sales_combined = pl.concat(df_prev_month_sales_list, how="vertical_relaxed")

                df_prev_month_sales_combined = df_prev_month_sales_combined.join(
                    df_historico_precios.select(['historico_precio_id', 'precio_base']),
                    on='historico_precio_id',
                    how='left'
                )

                monthly_summary = df_prev_month_sales_combined.group_by('producto_id').agg(
                    pl.sum('cantidad').alias('cantidad_total'),
                    pl.mean('precio_base').alias('precio_promedio')
                )

                for row in monthly_summary.iter_rows(named=True):
                    key = (row['producto_id'], prev_month_str_for_key)
                    last_product_monthly_state[key] = {
                        'precio': row['precio_promedio'],
                        'cantidad': row['cantidad_total']
                    }

        # 2. Calculate TARGET monthly quantities for each product for the *current* month
        monthly_product_target_quantities.clear() # Reset for the new month
        daily_product_sales_distribution.clear() # Reset for the new month

        days_in_current_month = (current_sale_date.replace(month=current_sale_date.month % 12 + 1, day=1) - timedelta(days=1)).day

        # Modificada la lógica de EPD y cambio mensual
        for prod_id in df_productos['producto_id'].to_list():
            prev_month_key = (prod_id, (current_sale_date - timedelta(days=current_sale_date.day)).replace(day=1).strftime('%Y-%m'))

            # Get previous month's actual total quantity and average price
            prev_month_total_quantity = last_product_monthly_state.get(prev_month_key, {'cantidad': 0})['cantidad']
            prev_month_avg_price = last_product_monthly_state.get(prev_month_key, {'precio': 0})['precio']

            # Si no hay datos del mes anterior, inicializar con una base razonable
            if prev_month_total_quantity == 0:
                base_target_quantity = (n_ventas / n_productos / total_days_simulation) * days_in_current_month * 2
            else:
                base_target_quantity = prev_month_total_quantity

            # Obtener precio actual para cálculo de EPD
            current_price_record = df_historico_precios.filter(
                (pl.col('producto_id') == prod_id) &
                (pl.col('fecha_actualizacion').cast(pl.Date) <= current_sale_date.date())
            ).sort('fecha_actualizacion', descending=True).head(1)

            current_prod_price = current_price_record['precio_base'].item() if not current_price_record.is_empty() else 100

            # Calcular cambio de precio y EPD
            porcentaje_cambio_precio = 0
            if prev_month_avg_price > 0:
                porcentaje_cambio_precio = (current_prod_price - prev_month_avg_price) / prev_month_avg_price

            categoria_producto = product_category_map.get(prod_id)
            rango_epd = elasticidades_por_categoria.get(categoria_producto, {'min': -1.0, 'max': -2.0})
            epd_objetivo = random.uniform(rango_epd['min'], rango_epd['max'])
            
            # Aplicar elasticidad positiva con menor probabilidad
            if random.random() < prob_elasticidad_positiva:
                epd_objetivo = random.uniform(rango_elasticidad_positiva['min'], rango_elasticidad_positiva['max'])

            # Calcular el efecto de la elasticidad
            epd_influenced_change = epd_objetivo * porcentaje_cambio_precio

            # Verificar promociones activas para el producto
            promociones_activas = df_promociones.filter(
                (pl.col('fecha_inicio').cast(pl.Date) <= current_sale_date.date()) &
                (pl.col('fecha_fin').cast(pl.Date) >= current_sale_date.date())
            )

            # Ajustar el efecto de la elasticidad si hay promociones
            if not promociones_activas.is_empty():
                descuento_promedio = promociones_activas['descuento_porcentaje'].mean() / 100
                promotion_boost = random.uniform(MIN_PROMOTION_BOOST, MAX_PROMOTION_BOOST)
                epd_influenced_change = epd_influenced_change * PROMOTION_ELASTICITY_MULTIPLIER * (1 + descuento_promedio)
                base_monthly_change_factor = promotion_boost

            # Combinar influencia de EPD con tendencia mensual base
            base_monthly_change_factor = random.uniform(0.95, 1.05) * (1 + epd_influenced_change)

            # Aplicar límites a los cambios base
            clipped_base_change_factor = np.clip(
                base_monthly_change_factor,
                1 - MAX_PERCENT_CHANGE_PER_MONTH_PRODUCT_AGGREGATE,
                1 + MAX_PERCENT_CHANGE_PER_MONTH_PRODUCT_AGGREGATE
            )

            # Aplicar multiplicador estacional
            current_seasonal_multiplier = seasonal_multipliers.get(month, 1.0)
            
            # Ajustar el multiplicador estacional según la categoría del producto
            if categoria_producto in ['Agua', 'Gaseosa', 'Bebida de Té']:
                # Productos más sensibles a la temporada cálida
                if month in [6, 7, 8]:  # Meses de verano
                    current_seasonal_multiplier *= 1.2
        
            # Productos más sensibles a la temporada navideña
            if month == 12:
                current_seasonal_multiplier *= 1.15

            final_monthly_change_factor = clipped_base_change_factor * current_seasonal_multiplier

            target_monthly_quantity = int(base_target_quantity * final_monthly_change_factor)
            target_monthly_quantity = max(1, target_monthly_quantity)

            monthly_product_target_quantities[(prod_id, current_month_str)] = target_monthly_quantity

        # 3. Distribute monthly target quantities across days of the current month
        # This will be used to guide daily sales generation
        for prod_id, total_monthly_q in monthly_product_target_quantities.items():
            if prod_id[1] != current_month_str: # Only process for the current month
                continue

            base_daily_q = total_monthly_q / days_in_current_month

            for d_offset_in_month in range(days_in_current_month):
                date_in_month = current_sale_date.replace(day=1) + timedelta(days=d_offset_in_month)
                daily_q = base_daily_q * random.uniform(1 - DAILY_QUANTITY_FLUCTUATION_FACTOR, 1 + DAILY_QUANTITY_FLUCTUATION_FACTOR)
                daily_product_sales_distribution[(prod_id[0], date_in_month.strftime('%Y-%m-%d'))] = max(1, int(daily_q))

        last_month_year = current_month_str

    # --- Daily Sales Generation based on Calculated Monthly Targets ---

    # Calculate total sales for today based on overall average and trend
    base_sales_per_day = n_ventas / total_days_simulation
    overall_daily_sales_num_multiplier = random.uniform(0.95, 1.05) # Small daily fluctuation for number of sales
    num_sales_today = int(base_sales_per_day * monthly_sales_trend_multiplier * overall_daily_sales_num_multiplier)
    num_sales_today = max(1, num_sales_today) # Ensure at least one sale

    daily_sales_data = []

    # Products and their remaining target quantities for TODAY
    current_day_product_targets = {
        prod_id: daily_product_sales_distribution.get((prod_id, current_sale_date_str), 0)
        for prod_id in df_productos['producto_id'].to_list()
    }

    # Create a list of products to consider for sales today, prioritizing those with higher remaining targets
    products_for_today_sales = [
        p_id for p_id, target_q in current_day_product_targets.items() if target_q > 0
    ]

    for _ in range(num_sales_today):
        product_id = None
        if products_for_today_sales: # If there are products with remaining daily targets
            remaining_quantities = np.array([current_day_product_targets[p_id] for p_id in products_for_today_sales])
            if remaining_quantities.sum() > 0:
                probabilities = remaining_quantities / remaining_quantities.sum()
                product_id = np.random.choice(products_for_today_sales, p=probabilities)
            else: # Fallback, should be rare if products_for_today_sales is non-empty
                product_id = random.choice(df_productos['producto_id'].to_list())
        else:
            # If all product daily targets are fulfilled for today, pick a random product from all products
            product_id = random.choice(df_productos['producto_id'].to_list())


        cliente_id = np.random.choice(df_clientes['cliente_id'])
        region_id = np.random.choice(regiones_para_seleccion, p=prob_regiones_para_seleccion)
        canal_id = np.random.choice(df_canales['canal_id'])

        # Lógica de promociones
        promociones_validas_hoy = df_promociones.filter(
            (pl.col('fecha_inicio').cast(pl.Date) <= current_sale_date.date()) &
            (pl.col('fecha_fin').cast(pl.Date) >= current_sale_date.date())
        )

        promocion_id = None
        if not promociones_validas_hoy.is_empty() and random.random() < 0.5:
            promocion_id = random.choice(promociones_validas_hoy['promocion_id'].to_list())

        # Búsqueda de precios
        precio_registro = df_historico_precios.filter(
            (pl.col('producto_id') == product_id) &
            (pl.col('fecha_actualizacion').cast(pl.Date) <= current_sale_date.date())
        ).sort('fecha_actualizacion', descending=True).head(1)

        if precio_registro.is_empty():
            # Fallback if no price found (should be rare with good price history generation)
            # You might want to log this or assign a default price if it occurs frequently
            continue # Skip this sale if no price can be determined for the product

        historico_precio_id = precio_registro['historico_precio_id'].item()

        # Determine quantity for this specific sale
        # Try to take a chunk from the remaining daily target
        quantity = random.randint(MIN_QUANTITY_PER_SALE, MAX_QUANTITY_PER_SALE)

        # If there's a daily target, adjust quantity to meet it
        if current_day_product_targets.get(product_id, 0) > 0:
            quantity_for_sale_attempt = random.randint(MIN_QUANTITY_PER_SALE, MAX_QUANTITY_PER_SALE)
            quantity = min(quantity_for_sale_attempt, current_day_product_targets[product_id])
            quantity = max(1, quantity) # Ensure at least 1 unit if target > 0

            current_day_product_targets[product_id] -= quantity
            if current_day_product_targets[product_id] <= 0:
                # Remove product from list only if its target is truly met
                if product_id in products_for_today_sales: # Check before removing
                    products_for_today_sales.remove(product_id)

        daily_sales_data.append({
            'venta_id': venta_id_counter_final,
            'fecha': current_sale_date_str,
            'cliente_id': cliente_id,
            'producto_id': product_id,
            'cantidad': quantity,
            'canal_id': canal_id,
            'region_id': region_id,
            'promocion_id': promocion_id,
            'historico_precio_id': historico_precio_id
        })
        venta_id_counter_final += 1
        
        # Check if total sales reached within the loop to break early if desired
        if venta_id_counter_final > n_ventas:
            break # Break from the daily sales loop if we have enough sales overall

    if daily_sales_data:
        sales_data_chunks.append(pl.DataFrame(daily_sales_data, schema=daily_sales_schema))
    
    # Check if total sales reached, and if so, break from the main daily loop
    if venta_id_counter_final > n_ventas:
        print(f"Objetivo de {n_ventas} ventas alcanzado. Deteniendo la generación de datos.")
        break


# Concatenación final de todas las ventas
# Use take(n_ventas) to ensure the total number of sales does not exceed the parameter
# Use slice to ensure exactly n_ventas if possible
df_ventas = pl.concat(sales_data_chunks, how="vertical_relaxed")
df_ventas = df_ventas.slice(0, n_ventas) # Get exactly n_ventas rows if available

# --- Guardar en CSV ---
df_clientes.write_csv('clientes.csv')
df_productos.write_csv('productos.csv')
df_historico_precios.write_csv('historico_precios.csv')
df_canales.write_csv('canales.csv')
df_regiones.write_csv('regiones.csv')
df_promociones.write_csv('promociones.csv')
df_inventarios.write_csv('inventarios.csv')
df_ventas.write_csv('ventas.csv')

print(f"\nConjuntos de datos generados y guardados como CSV.")
print(f"Total de ventas: {len(df_ventas)}")
print(f"Total de productos: {len(df_productos)}")
print(f"Total de entradas de historial de precios: {len(df_historico_precios)}")
print(f"Total de clientes: {len(df_clientes)}")
print(f"Total de regiones: {len(df_regiones)}")
print(f"Total de canales: {len(df_canales)}")
print(f"Total de promociones: {len(df_promociones)}")
print(f"Total de entradas de inventario: {len(df_inventarios)}")

# --- Opcional: Verificación de la variación mensual (para depuración) ---
df_ventas_temp = df_ventas.with_columns(pl.col('fecha').str.strptime(pl.Date, '%Y-%m-%d'))
monthly_product_sales = df_ventas_temp.group_by(['producto_id', pl.col('fecha').dt.year().alias('año'), pl.col('fecha').dt.month().alias('mes')]).agg(
    pl.sum('cantidad').alias('cantidad_total_mes')
).sort(['producto_id', 'año', 'mes'])

print("\nVerificación de variación mensual (primeros 5 productos):")
for product_id in range(1, 6): # Check for first 5 products
    product_sales = monthly_product_sales.filter(pl.col('producto_id') == product_id)
    if product_sales.height > 1:
        print(f"Producto {product_id}:")
        for i in range(1, product_sales.height):
            q_prev = product_sales[i-1, 'cantidad_total_mes']
            q_curr = product_sales[i, 'cantidad_total_mes']
            if q_prev > 0:
                change_percent = ((q_curr - q_prev) / q_prev) * 100
                print(f"  {product_sales[i-1, 'año']}-{product_sales[i-1, 'mes']} a {product_sales[i, 'año']}-{product_sales[i, 'mes']}: {change_percent:.2f}%")

Generando datos de ventas: 100%|██████████| 897/897 [08:26<00:00,  1.77it/s]




Conjuntos de datos generados y guardados como CSV.
Total de ventas: 199330
Total de productos: 40
Total de entradas de historial de precios: 1200
Total de clientes: 2000
Total de regiones: 10
Total de canales: 5
Total de promociones: 20
Total de entradas de inventario: 120

Verificación de variación mensual (primeros 5 productos):
Producto 1:
  2023-1 a 2023-2: -9.85%
  2023-2 a 2023-3: 17.27%
  2023-3 a 2023-4: 1.60%
  2023-4 a 2023-5: -3.70%
  2023-5 a 2023-6: 0.72%
  2023-6 a 2023-7: -17.13%
  2023-7 a 2023-8: -13.19%
  2023-8 a 2023-9: -4.10%
  2023-9 a 2023-10: 2.95%
  2023-10 a 2023-11: -2.29%
  2023-11 a 2023-12: 28.06%
  2023-12 a 2024-1: -20.25%
  2024-1 a 2024-2: 3.08%
  2024-2 a 2024-3: 22.55%
  2024-3 a 2024-4: 0.62%
  2024-4 a 2024-5: -1.41%
  2024-5 a 2024-6: -16.37%
  2024-6 a 2024-7: 3.22%
  2024-7 a 2024-8: -17.04%
  2024-8 a 2024-9: -17.27%
  2024-9 a 2024-10: 19.81%
  2024-10 a 2024-11: 5.08%
  2024-11 a 2024-12: 34.77%
  2024-12 a 2025-1: -3.42%
  2025-1 a 2025-2: 